In [1]:
# Necessary imports of functions and dictionaries
from dictionaries import *
from sample_sheet_functions import *
import pandas as pd
from io import StringIO



###### For future
- Give possibility to download/upload in CSV/XLSX
- Table shows string on tooltip
- Table captures with a regular expression the sample name prefixes


# User download for template
_User Required to_
- Pick the table flavours they want (in form of dictionary key:True/False)
- Add project name, experiment name and additional comments (Now all stored in sample sheet template)

In [2]:
# Essential input
plates_options = {'24': False, '48': False, 'e': False, 'f': False, 'g': False, 'h': False, 'i': False, 'j': False, 'k': False, 'l': False}
project_name = ''

# Non-essential
experiment_name = ''
additional_comments = ''

file_format=None #Needs to be done at some point

In [3]:
# User input simulation
user_project_name = 'user project'
user_experiment_name = 'this is my project'
user_additional_comments = 'i want more space for the comments!'

user_plate_selection = {'24': True, '48': False, 'e': True, 'f': True, 'g': False, 'h': False, 'i': False, 'j': False, 'k': False, 'l': True}

In [26]:
# 
# print(different_24_dict)


df1 = pd.DataFrame.from_dict(different_96_dict)
df1 


df1.to_csv('96_well_template.csv')

In [31]:
new_df1 = pd.read_csv('96_well_template.csv')
new_df1

,Unnamed: 0,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,R11,R12
0,F01,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
1,F02,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
2,F03,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
3,F04,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
4,F05,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
5,F06,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
6,F07,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
7,F08,sample,sample,sample,sample,sample,sample,_bsnc,_bspc,_bspc,_bspc,_bspc,_bsnc


In [32]:
# Handle 96 well plate

flavour = 'msfnoenfe'

new_cols = [x if not x.startswith('Unnamed') else flavour for x in new_df1.columns]
new_df1.columns = new_cols

new_df1


new_df1.to_csv('to_delete_now.csv', index=False)


,msfnoenfe,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,R11,R12
0,F01,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
1,F02,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
2,F03,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
3,F04,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
4,F05,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
5,F06,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
6,F07,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample,sample
7,F08,sample,sample,sample,sample,sample,sample,_bsnc,_bspc,_bspc,_bspc,_bspc,_bsnc


In [9]:

###################################################### FOR 24/48
re_import_df = pd.DataFrame.from_dict(different_48_dict)

re_import_df.to_csv('to_delete_now.csv', index=False)

